# Develop our pipeline

In this notebook, we will develop our Azure Machine Learning Pipeline. The Azure Machine learning pipeline will string together the steps of preprocessing the video, applying style transfer, and postprocessing the video into a single execution graph. 

To setup the pipeline, we'll need to make sure we have the necessary compute and storage available. To do so, we'll need to create our compute platform using AmlCompute and register the storage account that we created in the previous notebook.

The last step of this notebook is to publish the pipeline. Once it's published as a public endpoint, we'll test it to make sure that it runs as expected.

---

### Import package and load .env

In [33]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
from azureml.core import Workspace, Run, Experiment
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep, MpiStep
from azureml.core.runconfig import CondaDependencies, RunConfiguration
from azureml.core.runconfig import DEFAULT_CPU_IMAGE #, DEFAULT_GPU_IMAGE
from IPython.core.display import display, HTML
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.pipeline.core.graph import PipelineParameter
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core.authentication import AzureCliAuthentication
import subprocess
import requests
import json
import os

from MetricsUtils.hpStatisticsCollection import statisticsCollector, CollectionEntry
from MetricsUtils.storageutils import storageConnection

In [3]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Setup the workspace in AML

Get our workspace from the config file.

In [4]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# Also create a Project and attach to Workspace
project_folder = "scripts"
run_history_name = project_folder

if not os.path.isdir(project_folder):
    os.mkdir(project_folder)

Workspace name: jiata-bsdlaml-0
Azure region: southcentralus
Subscription id: 989b90f7-da4f-41f9-84c9-44848802052d
Resource group: jiata-bsdlaml-0


### Setup the compute

Create our compute using `AmlCompute`. We'll need one node for the video pre/post processing. And the remaining nodes for performing the style transfer. Since we'll be using the MPI Step, all nodes must be active before the MPI step will execute. Thus, we should set max nodes to equal min nodes, as there is no point autoscaling the cluster.

Set the number of nodes we want for each cluster.

In [5]:
style_transfer_node_count = 4
ffmpeg_node_count = 1

Verify that the subscription in use has enough cores. We need to check for two vm types since we'll be using NCSv2 for style transfer and DSv2 for ffmpeg processes. If you do not have quota for the NCSv2 family, you can use another GPU family instead.

In [6]:
vm_dict = {
    "NCSv2": {
        "size": "STANDARD_NC6s_v2",
        "cores": 6
    },
    "NCSv3": {
        "size": "STANDARD_NC6s_v3",
        "cores": 6
    },
    "DSv2": {
        "size": "STANDARD_DS3_V2",
        "cores": 4
    }
}

Create our non-gpu DSv2 cluster

In [7]:
# CPU compute
cpu_cluster_name = "ffmpeg-cluster"
try:
    cpu_cluster = AmlCompute(ws, cpu_cluster_name)
    print("Found existing cluster.")
except:
    print("Creating {}".format(cpu_cluster_name))
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size=vm_dict["DSv2"]["size"], 
        min_nodes=ffmpeg_node_count, 
        max_nodes=ffmpeg_node_count
    )

    # create the cluster
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_config)
    cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster.


Create our NCSv2 cluster.

In [8]:
# GPU compute
gpu_cluster_name = "style-cluster"
try:
    gpu_cluster = AmlCompute(ws, gpu_cluster_name)
    print("Found existing cluster.")
except:
    print("Creating {}".format(gpu_cluster_name))
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size=vm_dict["NCSv2"]["size"], 
        min_nodes=style_transfer_node_count, 
        max_nodes=style_transfer_node_count
    )

    # create the cluster
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, provisioning_config)
    gpu_cluster.wait_for_completion(show_output=True)

Found existing cluster.


### Setup data references

Create a datastore based on the storage account we created earlier. We'll use that storage account to hold our input and output data.

In [9]:
my_datastore_name = "datastore"
set_key(env_path, "AML_DATASTORE_NAME", my_datastore_name)

(True, 'AML_DATASTORE_NAME', 'datastore')

In [10]:
# datastore
my_datastore = Datastore.register_azure_blob_container(
    workspace=ws, 
    datastore_name=my_datastore_name, 
    container_name=get_key(env_path, "STORAGE_CONTAINER_NAME"), 
    account_name=get_key(env_path, "STORAGE_ACCOUNT_NAME"), 
    account_key=get_key(env_path, "STORAGE_ACCOUNT_KEY"),
    overwrite=True
)

Upload the `models` folder (from out local directory) and the `orangutan.mp4` video to the datastore.

In [11]:
!wget -O orangutan.mp4 https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/input_video.mp4

--2019-12-05 15:49:43--  https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/input_video.mp4
Resolving happypathspublic.blob.core.windows.net (happypathspublic.blob.core.windows.net)... 52.239.214.164
Connecting to happypathspublic.blob.core.windows.net (happypathspublic.blob.core.windows.net)|52.239.214.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7961293 (7.6M) [video/mp4]
Saving to: ‘orangutan.mp4’

orangutan.mp4       100%[===================>]   7.59M  14.5MB/s    in 0.5s    

2019-12-05 15:49:43 (14.5 MB/s) - ‘orangutan.mp4’ saved [7961293/7961293]



In [12]:
# Upload files in models folder to a directory called models
my_datastore.upload_files(
    ["./models/model.pth"],
    target_path="models", 
    overwrite=True
)

# Upload orangutan.mp4 video
my_datastore.upload_files(
    ["./orangutan.mp4"],
    overwrite=True
)

Uploading an estimated of 1 files
Uploading ./models/model.pth
Uploaded ./models/model.pth, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading ./orangutan.mp4
Uploaded ./orangutan.mp4, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_datastore

Set the `models` dir we uploaded as data references to be used by the pipeline steps later on.

In [13]:
model_dir = DataReference(
    data_reference_name="model_dir", 
    datastore=my_datastore, 
    path_on_datastore="models", 
    mode="download"
)

Set the output video to be saved in the same datastore.

In [14]:
output_video = PipelineData(name="output_video", datastore=my_datastore)

Get a reference to the datastore that was generated when the AML workspace was created. We'll use this datastore to hold temporary pipeline data.

In [15]:
default_datastore = ws.get_default_datastore()     

Save all temporary data files (PipelineData) to the default datastore.

In [16]:
ffmpeg_audio = PipelineData(name="ffmpeg_audio", datastore=default_datastore)
ffmpeg_images = PipelineData(name="ffmpeg_images", datastore=default_datastore)
processed_images = PipelineData(name="processed_images", datastore=default_datastore)

### Setup cluster environments

Config for ffmpeg cluster

In [19]:
ffmpeg_cd = CondaDependencies()
ffmpeg_cd.add_channel("conda-forge")
ffmpeg_cd.add_conda_package("ffmpeg")

ffmpeg_run_config = RunConfiguration(conda_dependencies=ffmpeg_cd)
ffmpeg_run_config.environment.docker.enabled = True
ffmpeg_run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
ffmpeg_run_config.environment.spark.precache_packages = False

Config for style transfer cluster

In [20]:
style_transfer_cd = CondaDependencies()
style_transfer_cd.add_channel("pytorch")
style_transfer_cd.add_conda_package("pytorch")

style_transfer_run_config = RunConfiguration(conda_dependencies=style_transfer_cd)
style_transfer_run_config.environment.docker.enabled = True
style_transfer_run_config.environment.docker.base_image = "pytorch/pytorch"
style_transfer_run_config.environment.spark.precache_packages = False

### Set up pipeline steps

When setting up the pipelines, we'll need to create a `video_path_param` that can be modified when the pipeline is published.

In [21]:
video_path_default = DataPath(datastore=my_datastore, path_on_datastore="orangutan.mp4")
video_path_param = (PipelineParameter(name="video_path", default_value=video_path_default), DataPathComputeBinding())

Create the 3-step pipeline using PythonScriptSteps and the MpiStep. In the MPI step, you'll notice that we use the `style_transfer_mpi.py` script instead of the `style_transfer.py` script. This is because the MPI expects that the script is modified to use MPI code.

Both scripts do the exact same thing, except that the `style_transfer_mpi.py` script is set up to use MPI to run process the frames in a distributed way. 

Feel free to inspect the differences under the `scripts` folder.

In [22]:
preprocess_video_step = PythonScriptStep(
    name="preprocess video",
    script_name="preprocess_video.py",
    arguments=["--input-video", video_path_param,
               "--output-audio", ffmpeg_audio,
               "--output-images", ffmpeg_images,
              ],
    compute_target=cpu_cluster,
    inputs=[video_path_param],
    outputs=[ffmpeg_images, ffmpeg_audio],
    runconfig=ffmpeg_run_config,
    source_directory=project_folder,
    allow_reuse=False
)

distributed_style_transfer_step = MpiStep(
    name="mpi style transfer",
    script_name="style_transfer_mpi.py",
    arguments=["--content-dir", ffmpeg_images,
               "--output-dir", processed_images,
               "--model-dir", model_dir,
               "--cuda", 1
              ],
    compute_target=gpu_cluster,
    node_count=4, 
    process_count_per_node=1,
    inputs=[model_dir, ffmpeg_images],
    outputs=[processed_images],
    pip_packages=["image", "mpi4py", "torch", "torchvision"],
    runconfig=style_transfer_run_config,
    use_gpu=True,
    source_directory=project_folder,
    allow_reuse=False
)

postprocess_video_step = PythonScriptStep(
    name="postprocess video",
    script_name="postprocess_video.py",
    arguments=["--images-dir", processed_images, 
               "--input-audio", ffmpeg_audio, 
               "--output-dir", output_video],
    compute_target=cpu_cluster,
    inputs=[processed_images, ffmpeg_audio],
    outputs=[output_video],
    runconfig=ffmpeg_run_config,
    source_directory=project_folder,
    allow_reuse=False
)

WARNING - 'distributed_backend' parameter will be deprecated. Please use 'distributed_training' instead.


### Run the pipeline

Run the pipeline, passing in the video path variable.

In [23]:
steps = [postprocess_video_step]
pipeline = Pipeline(workspace=ws, steps=steps)
pipeline_run = Experiment(ws, 'style_transfer_mpi').submit(
    pipeline, 
    pipeline_parameters={'video_path': DataPath(datastore=my_datastore, path_on_datastore="orangutan.mp4")}
)

WARNING - The 'pipeline_params' argument is deprecated. Please use 'pipeline_parameters' instead.


Created step postprocess video [5b1095ed][b63de06d-3e9e-4f7f-bb8b-fd6f9c0e3892], (This step will run and generate new outputs)Created step mpi style transfer [81c81fb1][fac54982-954d-474c-9a3e-98d0956f0e77], (This step will run and generate new outputs)

Created step preprocess video [1ce16f06][262a5745-6ee6-4f52-b31c-3c43d22751a7], (This step will run and generate new outputs)
Using data reference model_dir for StepId [d4d90692][e9d8c902-6ad2-4eee-a600-7c05c08bfe4c], (Consumers of this data are eligible to reuse prior runs.)
Created data reference datastore_f8ff6269 for StepId [bbec8e46][36a8d371-06f8-448b-b69d-ce3f2388674a], (Consumers of this data will generate new runs.)
Submitted PipelineRun 38c3f15c-e618-44e8-8f19-cd06ec19998f
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/style_transfer_mpi/runs/38c3f15c-e618-44e8-8f19-cd06ec19998f?wsid=/subscriptions/989b90f7-da4f-41f9-84c9-44848802052d/resourcegroups/jiata-bsdlaml-0/workspaces/jiata-bsdlaml-0


In [24]:
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
style_transfer_mpi,38c3f15c-e618-44e8-8f19-cd06ec19998f,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Wait until the pipeline completes before proceeding...

In [25]:
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 38c3f15c-e618-44e8-8f19-cd06ec19998f
Link to Portal: https://ml.azure.com/experiments/style_transfer_mpi/runs/38c3f15c-e618-44e8-8f19-cd06ec19998f?wsid=/subscriptions/989b90f7-da4f-41f9-84c9-44848802052d/resourcegroups/jiata-bsdlaml-0/workspaces/jiata-bsdlaml-0
PipelineRun Status: Running


StepRunId: 1d72e487-2529-450a-9ac6-853e4eb10ce3
Link to Portal: https://ml.azure.com/experiments/style_transfer_mpi/runs/1d72e487-2529-450a-9ac6-853e4eb10ce3?wsid=/subscriptions/989b90f7-da4f-41f9-84c9-44848802052d/resourcegroups/jiata-bsdlaml-0/workspaces/jiata-bsdlaml-0
StepRun( preprocess video ) Status: NotStarted
StepRun( preprocess video ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_2e7a5e04f6b9a47fce3ffb2d351ff4cd81ca68a0ad9f6fac516909d29f5e81b9_d.txt
2019-12-05T15:53:45Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_e7c61fe000f77e9812d4681c2e711220
Digest: sha256:12c32fb6377897d20dda3ea3297847aa

frame=  353 fps= 43 q=24.8 size=N/A time=00:00:11.76 bitrate=N/A speed=1.43x    
frame=  374 fps= 43 q=24.8 size=N/A time=00:00:12.46 bitrate=N/A speed=1.43x    
frame=  395 fps= 43 q=24.8 size=N/A time=00:00:13.16 bitrate=N/A speed=1.43x    
frame=  418 fps= 43 q=24.8 size=N/A time=00:00:13.93 bitrate=N/A speed=1.43x    
frame=  441 fps= 43 q=24.8 size=N/A time=00:00:14.70 bitrate=N/A speed=1.44x    
frame=  463 fps= 43 q=24.8 size=N/A time=00:00:15.43 bitrate=N/A speed=1.44x    
frame=  485 fps= 43 q=24.8 size=N/A time=00:00:16.16 bitrate=N/A speed=1.44x    
frame=  508 fps= 43 q=24.8 size=N/A time=00:00:16.93 bitrate=N/A speed=1.44x    
frame=  530 fps= 43 q=24.8 size=N/A time=00:00:17.66 bitrate=N/A speed=1.44x    
frame=  553 fps= 43 q=24.8 size=N/A time=00:00:18.43 bitrate=N/A speed=1.44x    
frame=  574 fps= 43 q=24.8 size=N/A time=00:00:19.13 bitrate=N/A speed=1.44x    
frame=  596 fps= 43 q=24.8 size=N/A time=00:00:19.86 bitrate=N/A speed=1.44x    
frame=  618 fps= 43 q=24.8 s




StepRunId: dc7bacaa-87e9-44d6-9627-8b60a7d87ee5
Link to Portal: https://ml.azure.com/experiments/style_transfer_mpi/runs/dc7bacaa-87e9-44d6-9627-8b60a7d87ee5?wsid=/subscriptions/989b90f7-da4f-41f9-84c9-44848802052d/resourcegroups/jiata-bsdlaml-0/workspaces/jiata-bsdlaml-0
StepRun( mpi style transfer ) Status: NotStarted
StepRun( mpi style transfer ) Status: Queued
StepRun( mpi style transfer ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_40e2d4c4eaec4d9a0d1e6db7a0d07cabc36aaadc033e33b90618b526745b9fd6_d.txt
2019-12-05T15:56:18Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_ddda1ac8b76b965a070517439825a791
Digest: sha256:7d2d5977ae34b328339a39e647ea0f66b6ffddba343daaf2e238e4e261cd99c1
Status: Image is up to date for jiatabsdlaml65909f7d.azurecr.io/azureml/azureml_ddda1ac8b76b965a070517439825a791:latest
aa7402a306f28ec5a72ee285573b9cd3b1b6b0545231fdf37fc1731049a73494
2019/12/05 15:56:20 Version: 3.0.0105




StepRunId: 9d563b41-488d-476c-a5d7-207ea913ed82
Link to Portal: https://ml.azure.com/experiments/style_transfer_mpi/runs/9d563b41-488d-476c-a5d7-207ea913ed82?wsid=/subscriptions/989b90f7-da4f-41f9-84c9-44848802052d/resourcegroups/jiata-bsdlaml-0/workspaces/jiata-bsdlaml-0
StepRun( postprocess video ) Status: NotStarted
StepRun( postprocess video ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_2e7a5e04f6b9a47fce3ffb2d351ff4cd81ca68a0ad9f6fac516909d29f5e81b9_d.txt
2019-12-05T15:59:23Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_e7c61fe000f77e9812d4681c2e711220
Digest: sha256:12c32fb6377897d20dda3ea3297847aa30799dfa51eb4ccef98d0c997337643c
Status: Image is up to date for jiatabsdlaml65909f7d.azurecr.io/azureml/azureml_e7c61fe000f77e9812d4681c2e711220:latest
1318e4b9ef2697247a749d39c68ae7349bb6b4958facceea67fb9005d8f7bd1c
2019/12/05 15:59:25 Version: 3.0.01059.0002 Branch: master Commit: e8f402a4
2019/12/

frame=  422 fps= 29 q=26.0 size=   36864kB time=00:00:12.30 bitrate=24551.9kbits/s speed=0.833x    
frame=  437 fps= 29 q=26.0 size=   38400kB time=00:00:12.80 bitrate=24575.9kbits/s speed=0.836x    
frame=  452 fps= 29 q=26.0 size=   39936kB time=00:00:13.30 bitrate=24598.1kbits/s speed=0.84x    
frame=  466 fps= 28 q=26.0 size=   41216kB time=00:00:13.76 bitrate=24525.9kbits/s speed=0.841x    
frame=  481 fps= 28 q=26.0 size=   42752kB time=00:00:14.26 bitrate=24548.4kbits/s speed=0.845x    
frame=  495 fps= 28 q=26.0 size=   44032kB time=00:00:14.73 bitrate=24482.5kbits/s speed=0.847x    
frame=  508 fps= 28 q=26.0 size=   45312kB time=00:00:15.16 bitrate=24474.4kbits/s speed=0.848x    
frame=  523 fps= 28 q=26.0 size=   46848kB time=00:00:15.66 bitrate=24496.4kbits/s speed=0.852x    
frame=  538 fps= 28 q=26.0 size=   48384kB time=00:00:16.16 bitrate=24517.1kbits/s speed=0.853x    
frame=  552 fps= 28 q=26.0 size=   49664kB time=00:00:16.63 bitrate=24459.7kbits/s speed=0.853x    
f

{'runId': '9d563b41-488d-476c-a5d7-207ea913ed82', 'target': 'ffmpeg-cluster', 'status': 'Completed', 'startTimeUtc': '2019-12-05T15:59:22.557975Z', 'endTimeUtc': '2019-12-05T16:00:48.134289Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': '4a914ad8-cb4d-432d-93bb-f8c292764e44', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': '38c3f15c-e618-44e8-8f19-cd06ec19998f', '_azureml.ComputeTargetType': 'batchai', 'AzureML.DerivedImageName': 'azureml/azureml_e7c61fe000f77e9812d4681c2e711220', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'runDefinition': {'script': 'postprocess_video.py', 'arguments': ['--images-dir', '$AZUREML_DATAREFERENCE_processed_images', '--input-audio', '$AZUREML_DATAREFERENCE_ffmpeg_audio', '--output-dir', '$AZUREML_DATAREFERENCE_output_video'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicato

'Finished'

### Download the output video

Get the step id of the postprocessing step

In [26]:
step_id = pipeline_run.find_step_run("postprocess video")[0].id

Download the output files from the postprocessing step

In [27]:
my_datastore.download(
    target_path="aml_test_orangutan", 
    prefix=step_id, 
)

2

Display the generated output video that we just downloaded

In [28]:
display(HTML("""
    <video width="320" height="240" controls>
        <source src="aml_test_orangutan/{}/output_video/video_processed.mp4" type="video/mp4">
    </video>
""".format(step_id)))

### Publish the pipeline

The last step is to publish the pipeline so that the pipeline can be triggered on an http endpoint. We'll use Logic Apps in the next notebook to consume this endpoint.

In [29]:
published_pipeline = pipeline.publish(
    name="style transfer", 
    description="some description"
)

In [30]:
published_pipeline_id = published_pipeline.id
set_key(env_path, "AML_PUBLISHED_PIPELINE_ID", published_pipeline_id)

(True, 'AML_PUBLISHED_PIPELINE_ID', 'b5267c9a-fb93-44dd-b683-b3131f5e326d')

### Test the published pipeline

In [31]:
# cli_auth = AzureCliAuthentication()
# aad_token = cli_auth.get_authentication_header()

# response = requests.post(
#     published_pipeline.endpoint, 
#     headers=aad_token, 
#     json={
#         "ExperimentName": "My_Pipeline",
#         "DataPathAssignments": {
#             "video_path": {"DataStoreName": my_datastore_name,
#                            "RelativePath": "orangutan.mp4"}
#         }
#     }
# )

# run_id = response.json()["Id"]
# print(run_id)

### Schedule the published pipeline to run regularly

This step shows how to schedule the published pipeline to run regularly. This will also submit an initial run since a starting time for the schedule is not supplied.

In [34]:
schedule_name = "style_transfer"
experiment_name = "style_transfer_mpi"
frequency = "Hour"
interval = 1
recurrence = ScheduleRecurrence(frequency=frequency, interval=interval)
schedule = Schedule.create(
    workspace=ws,
    name=schedule_name,
    pipeline_id=published_pipeline.id,
    experiment_name=experiment_name,
    recurrence=recurrence,
    description=schedule_name
)

### Write out URI

Write the URI to the statistics tracker.

In [35]:
statisticsCollector.addEntry(
    CollectionEntry.AKS_REALTIME_ENDPOINT,
    published_pipeline.endpoint
)

Get a connection string to the workspace's storage to use to save the statistics.



In [36]:
storageConnString = "YOUR_STORAGE_CONNECTION_STRING"

In [37]:
if storageConnString == "YOUR_STORAGE_CONNECTION_STRING":
    resource_group = ws.resource_group
    stgAcctName = ws.get_details()['storageAccount'].split('/')[-1]
    storageConnString = storageConnection.getConnectionStringWithAzCredentials(resource_group, stgAcctName)
print("storage_conn_string: {}".format(storageConnString))

storage_conn_string: DefaultEndpointsProtocol=https;AccountName=jiatabsdstorage624362c64;AccountKey=CnjYRGRhWyRrvRIeM633puHpzmFeVKzE5gFbUS+5J/xf9fsdE8+bG3qe62Q0XxGLPuSMIRRZ/NHpZ7smCG5dbw==;EndpointSuffix=core.windows.net


Save the statistics collected so far.

In [38]:
statisticsCollector.uploadContent(storageConnString)

---

You are now ready to move on to the [next notebook](04_deploy_logic_apps.ipynb).